# 시세DB 구축 및 시세조회 API 개발

- 해당 작업을 위해 서버에 mysql을 설치 후 파이썬에서 클라이언트로써 접속한다.

## MariaDB 설치 후 클라이언트 접속 확인

In [16]:
import pymysql

In [17]:
kwargs = dict(
    host = "localhost",
    port = 3306,
    db = "Investar",
    user = "spark",
    passwd = 'Rnflsso34#',
    # autocommit = True,
    charset='utf8'
)

In [18]:
connection = pymysql.connect(**kwargs)

In [4]:
cursor = connection.cursor()
cursor.execute("SELECT version();")
result = cursor.fetchone()

print(f"MariaDB version: {result}")

connection.close()

MariaDB version: ('8.0.26-0ubuntu0.20.04.3',)


## 필요한 테이블 생성

In [12]:
connection = pymysql.connect(**kwargs)

In [19]:
cursor = connection.cursor()

In [26]:
## 회사별 종목코드 및 이름 등 정보 테이블
q1 = """
CREATE TABLE IF NOT EXISTS company_info(
    code VARCHAR(20),
    company VARCHAR(20),
    last_update DATE,
    PRIMARY KEY (code)
);
"""

## 각 종목코드별 OHLC 및 거래량 주가정보 테이블
q2 = """
CREATE TABLE IF NOT EXISTS daily_price(
    code VARCHAR(20),
    date DATE,
    open BIGINT(20),
    high BIGINT(20),
    low BIGINT(20),
    close BIGINT(20),
    diff BIGINT(20),
    volumne BIGINT(20),
    PRIMARY KEY (code, date)
);
"""

In [27]:
# 테이블 생성
cursor.execute(q1)
cursor.execute(q2)

0

In [7]:
cursor.execute("SHOW TABLES;")
rows = cursor.fetchall()
for row in rows:
    print(row)

In [20]:
## 생성한 테이블 삭제
q1 = """
DROP TABLE company_info, daily_price;
"""

cursor.execute(q1)
cursor.execute("SHOW TABLES;")
rows = cursor.fetchall()
for row in rows:
    print(row)

In [21]:
connection.close()

## DBUpdater.py 실행 후 테이블 확인

In [1]:
import pymysql
import pandas as pd

In [2]:
kwargs = dict(
    host = "localhost",
    port = 3306,
    db = "Investar",
    user = "spark",
    passwd = 'Rnflsso34#',
    # autocommit = True,
)

In [3]:
connection = pymysql.connect(**kwargs)

In [4]:
query = """
SELECT * FROM company_info;
"""
res = pd.read_sql(query, connection)
res

DatabaseError: Execution failed on sql '
SELECT * FROM company_info;
': (1146, "Table 'Investar.company_info' doesn't exist")

In [5]:
query = """
SELECT * FROM daily_price;
"""
res = pd.read_sql(query, connection)
res

,code,date,open,high,low,close,diff,volume


In [5]:
import sys

In [6]:
sys.path.append("../source/Investar/")

In [7]:
import DBUpdater

In [8]:
dbu = DBUpdater.DBUpdater()

In [9]:
dbu.execute_daily()

[2021-10-02 17:56] #0001 REPLACE INTO company_info VALUES (004840, DRB동일, 2021-10-02)
[2021-10-02 17:56] #0002 REPLACE INTO company_info VALUES (155660, DSR, 2021-10-02)
[2021-10-02 17:56] #0003 REPLACE INTO company_info VALUES (078930, GS, 2021-10-02)
[2021-10-02 17:56] #0004 REPLACE INTO company_info VALUES (001250, GS글로벌, 2021-10-02)
[2021-10-02 17:56] #0005 REPLACE INTO company_info VALUES (294870, HDC현대산업개발, 2021-10-02)
[2021-10-02 17:56] #0006 REPLACE INTO company_info VALUES (011200, HMM, 2021-10-02)
[2021-10-02 17:56] #0007 REPLACE INTO company_info VALUES (092220, KEC, 2021-10-02)
[2021-10-02 17:56] #0008 REPLACE INTO company_info VALUES (016380, KG동부제철, 2021-10-02)
[2021-10-02 17:56] #0009 REPLACE INTO company_info VALUES (001390, KG케미칼, 2021-10-02)
[2021-10-02 17:56] #0010 REPLACE INTO company_info VALUES (058860, KTis, 2021-10-02)
[2021-10-02 17:56] #0011 REPLACE INTO company_info VALUES (011070, LG이노텍, 2021-10-02)
[2021-10-02 17:56] #0012 REPLACE INTO company_info VALUES (

KeyboardInterrupt: 

In [11]:
dbu.codes

{}

In [21]:
tmp = dbu.read_naver('214420', '토니모리', 1)

Exception occured : No tables found


In [22]:
tmp

In [23]:
code = '214420'
company = "토니모리"
pages_to_fetch = 10

In [28]:
import urllib
from bs4 import BeautifulSoup

In [32]:
pg_url

'http://finance.naver.com/item/sise_day.nhn?code=214420&page=1'

In [31]:
pd.read_html(pg_url, header=0)

ValueError: No tables found

In [29]:
url = f"http://finance.naver.com/item/sise_day.nhn?code={code}"
headers = ("User-Agent", "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36")
opener = urllib.request.build_opener()
opener.addheaders = [headers]
with opener.open(url) as doc:
    if doc is None:
        print("None")
    html = BeautifulSoup(doc, 'lxml')
    pgrr = html.find('td', class_='pgRR')
    if pgrr is None:
        print("None")
    s = str(pgrr.a['href']).split('=')
    last_page = s[-1]
df = pd.DataFrame()
pages = min(int(last_page), pages_to_fetch)
for page in range(1, pages + 1):
    pg_url = '{}&page={}'.format(url, page)
    df = df.append(pd.read_html(pg_url, header=0)[0])
    tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
    print('[{}] {} ({}) : {:04d}/{:04d} pages are downloading...'.format(tmnow, company, code, page, pages), end="\r")


ValueError: No tables found

In [22]:
dbu.read_krx_code()

,code,company
0,004840,DRB동일
1,155660,DSR
2,078930,GS
3,001250,GS글로벌
4,294870,HDC현대산업개발
...,...,...
2457,318660,타임기술
2458,199800,툴젠
2459,222670,플럼라인생명과학
2460,331660,한국미라클피플사
